In [ ]:
from src.data.synthetic_data import SyntheticDataGenerator

generator = SyntheticDataGenerator(
    number_of_nonoutput_species=1, 
    reaction_rate_ranges={'k_f': (0, 1.0), 'k_r': (0, 1.0)},
    initial_concentration_range = (1, 10)
    )  


In [ ]:
generator.print_possible_sets()